In [ ]:
import os
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib
import math
import numpy as np
import random
import time
from tqdm import tnrange, tqdm_notebook
from struct import *
from tensorflow.examples.tutorials.mnist import input_data
from pyswarm import pso
from sklearn.decomposition import PCA
%matplotlib notebook
plt.rcParams['figure.figsize'] = (8, 10)

In [ ]:
def cart2pol(x, y):
    rho = np.sqrt(x**2 + y**2)
    phi = np.arctan2(y, x)
    return (rho, phi)

def pol2cart(rho, phi):
    x = rho * np.cos(phi)
    y = rho * np.sin(phi)
    return (x, y)

In [ ]:
def plot_codes(x, y, numbers, name):
    %matplotlib inline
    plt.rcParams['figure.figsize'] = (12, 12)
    plt.scatter(x, y, color=color_map[numbers])
    for i in range(10):
        index = np.argmax(numbers == i)
        x_pos = x[index]
        y_pos = y[index]
        plt.scatter(x_pos, y_pos, color=color_map[i], label=i)
    legend = plt.legend()
    plt.savefig(name+'.png')

In [ ]:
def compare_results(result, reference, shape, i):
    result = np.reshape(result, shape)
    reference = np.reshape(reference, shape)
    plt.subplot(1,2,1)
    plt.imshow(reference, cmap='gray')
    plt.subplot(1,2,2)
    plt.imshow(result, cmap='gray')
    plt.savefig(str(i)+'.png')
    plt.show()

In [ ]:
def plt_error(fig, ax, data_x, data_y):
    if ax[0].lines:
        for i in range(len(ax[0].lines)):
            ax[0].lines[i].set_xdata(data_x[-50:])
            ax[0].lines[i].set_ydata(data_y[i][-50:])
        ax[0].set_xlim(min(data_x[-50:]), max(data_x[-50:]))
        ax[0].set_ylim(0.9*np.min([data[-50:] for data in data_y]),
                       1.1*np.max([data[-50:] for data in data_y]))
    else:
        for line in data_y:
            ax[0].plot(data_x, line)
    if ax[1].lines:
        for i in range(len(ax[1].lines)):
            ax[1].lines[i].set_xdata(data_x[1:])
            ax[1].lines[i].set_ydata(data_y[i][1:])
        ax[1].set_xlim(data_x[1], data_x[-1])
        ax[1].set_ylim(0, np.max([data[1:] for data in data_y]))
    else:
        for line in data_y:
            ax[1].plot(data_x, line)
    fig.canvas.draw()

In [ ]:
def get_mask(layers, iterator):
    true_count = min(15, layers[iterator])
    false_count = max(0, layers[iterator] - true_count)  
    return tf.Variable(
            [
            tf.random_shuffle(
                [True for _ in range(true_count)] + [False for _ in range(false_count)]
            ) for i in range(layers[iterator + 1])
        ]
        )

In [ ]:
def generate_initial_value(layers, iterator):
    true_count = min(15, layers[iterator])
    false_count = max(0, layers[iterator] - true_count)  
    
    value = [np.random.permutation(np.concatenate([np.random.normal(0, 1, true_count), [0]*false_count])).tolist() for _ in range(layers[iterator + 1])]

    return  np.array(value).astype('float32')

In [ ]:
def selu(x, a = 1.6733, l = 1.0507):
    pos = l * tf.nn.relu(x)
    neg = l * (a * tf.exp( (x - tf.abs(x)) * 0.5 ) - a)
    return pos + neg

In [ ]:
def create_autoencoder_model(layers, x, activation, last_layer = 'linear'):
    size = (len(layers) - 1 ) * 2
    W = [None for _ in range(size)]
    y = [None for _ in range(size)]
    mask = [None for _ in range(size)]
    batch_size = tf.shape(x)[0]
    batch_tiled = tf.fill([batch_size, 1], 1.0)
        
    for iterator in range(len(layers) - 1): 
        if (activation == tf.nn.sigmoid):
            W[iterator] = tf.Variable(
                tf.concat([np.transpose(generate_initial_value(layers, iterator)), tf.zeros([1, layers[iterator + 1]])], 0), name= "W" + str(iterator)
            )
        elif (activation == selu):            
            W[iterator] = tf.Variable(
                tf.concat([
                    tf.truncated_normal([layers[iterator], layers[iterator + 1]], 0, 1/np.sqrt(layers[iterator])),
                    tf.zeros([1, layers[iterator + 1]])
                ], 0),
                name= "W" + str(iterator)
            )
        if iterator == 0:
            y[iterator] = activation(tf.matmul(tf.concat([x, batch_tiled], 1), W[iterator]))
        elif iterator == len(layers) - 2:
            y[iterator] = tf.matmul(tf.concat([y[iterator-1], batch_tiled], 1), W[iterator])
        else:
            y[iterator] = activation(tf.matmul(tf.concat([y[iterator-1], batch_tiled], 1), W[iterator]))
        
    for iterator in range(len(layers) - 1):
        if (activation == tf.nn.sigmoid):
            W[iterator + len(layers) - 1] = tf.Variable(
                tf.concat([
                    generate_initial_value(layers, len(layers) - 2 - iterator),
                    tf.zeros([1, layers[len(layers) - iterator - 2]])
                ], 0),    
                name= "W" + str(iterator + len(layers) - 1)
            )
 
        elif (activation == selu):
            W[iterator + len(layers) - 1] = tf.Variable(
                tf.concat([
                    tf.truncated_normal(
                        [layers[len(layers) - iterator - 1],
                         layers[len(layers) - iterator - 2]],
                        0, 
                        1/np.sqrt(layers[len(layers) - iterator - 1])),
                    tf.zeros([1, layers[len(layers) - iterator - 2]])
                ], 0),
                name= "W" + str(iterator + len(layers) - 1)
            )
        y[iterator + len(layers) - 1] = activation(tf.matmul(tf.concat([y[iterator + len(layers) - 2], batch_tiled], 1) , W[iterator + len(layers) - 1]))
    
    if(last_layer == 'linear'):
        y[size - 1] = tf.matmul(tf.concat([y[size - 2], batch_tiled], 1), W[size - 1])
    elif(last_layer == 'sigmoid'):
        y[size - 1] = tf.nn.sigmoid(tf.matmul(tf.concat([y[size - 2], batch_tiled], 1), W[size - 1]))
    elif(last_layer == 'selu'):
        y[size - 1] = selu(tf.matmul(tf.concat([y[size - 2], batch_tiled], 1), W[size - 1]))
    elif(last_layer == 'activation'):
        y[size - 1] = activation(tf.matmul(tf.concat([y[size - 2], batch_tiled], 1), W[size - 1]))
    else:
        y[size - 1] = activation(tf.matmul(tf.concat([y[size - 2], batch_tiled], 1), W[size - 1]))
    
    return W, y[-1], y[len(layers) - 2], y

In [ ]:
class DataSet:
    input_size = 0
    output_size = 0
    samples_count = 0
    inputs = []
    outputs = []
    double_size = 8
    min_value = 0
    max_value = 0

    def __init__(self, file_path, zero_one):
        self.read_data_bin(file_path, zero_one)
        
    def scale_to_zero_one(self, data):
        data = (data - self.min_value)/(self.max_value-self.min_value)
        return data
    
    def scale_to_original(self, data):
        return data*(self.max_value-self.min_value) + self.min_value
    
    def read_data_bin(self,file_path, zero_one):
        inputs = []
        outputs = []
        file = open(file_path, "rb")
        self.samples_count, self.input_size, self.output_size = unpack("iii", file.read(12))
                
        for i in tnrange(self.samples_count):
            self.inputs.append(unpack(str(self.input_size)+"d", file.read(self.double_size * self.input_size)))
            self.outputs.append(unpack(str(self.output_size)+"d", file.read(self.double_size * self.output_size)))

        self.inputs = np.array(self.inputs)
        self.outputs = np.array(self.outputs)

        self.min_value = self.inputs.min()        
        self.max_value = self.inputs.max()
        if(zero_one):
            self.inputs = self.scale_to_zero_one(self.inputs)
        
        file.close()
    
    def get_next_bach(self, count):
        indexes = random.sample(range(self.samples_count), count)
        return [self.inputs[indexes], self.outputs[indexes]]
    
    def get_full_data(self):
        return [self.inputs, self.outputs]                 

In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

In [ ]:
data_set = DataSet("C:\Mgr\DataSets\HandwrittenDigitsMnist.bin", True)

In [ ]:
activation = selu

input_dim = 784
autoencoder_layers = [784, 1000, 500, 250, 2]

shape = (28, 28)
last_layer = 'sigmoid'


x = tf.placeholder(tf.float32, [None, input_dim], name = 'x')

W, y, code_layer, layers_y = create_autoencoder_model(autoencoder_layers, x, activation, last_layer)

y_ = tf.placeholder(tf.float32, [None, input_dim])

In [ ]:
error_avg_sqr = tf.reduce_mean(tf.reduce_sum(tf.square(y_ - y), axis=1))
error_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y) + (1 - y_) * tf.log(1-y), axis=1))

In [ ]:
batch_xs, batch_ys = data_set.get_next_bach(200)

In [ ]:
moment = tf.placeholder(tf.float32, shape=[])
eps = tf.placeholder(tf.float32, shape=[])

grad_norms = [tf.Variable(1.0) for _ in range((len(autoencoder_layers)-1)*2)]

In [ ]:
opimalizer = tf.train.MomentumOptimizer(eps, moment, use_nesterov = False)
gradients = opimalizer.compute_gradients(error_entropy)
iteration = tf.placeholder(tf.float32, shape=[], name = 'iteration')

In [ ]:
train_step = opimalizer.apply_gradients(gradients)

In [ ]:
time1 = time.time()
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
time2 = time.time()

print((time2-time1))

In [ ]:
%matplotlib notebook

batch_size = 200 
print_info_rate = 10000      
iteration_count = 1_000_000    
max_moment = 0.99

eps_param = 0.0001

range_moment = 50_000

fig, ax = plt.subplots(2,1)

iterations = []
errors = []
mean_errors = []
mean_train_error_avg_sqr = 0
error_alpha = 0.2

grad_norms_agregated = [[] for _ in range(len(grad_norms))]

for i in tnrange(0, iteration_count+1, batch_size, desc='Learning'):
    batch_xs, batch_ys = data_set.get_next_bach(batch_size)
    momentum = 1 - max(1 - max_moment, 0.5 / (i//range_moment + 1))
    sess.run([train_step], feed_dict={x: batch_xs, y_: batch_xs, moment: momentum, eps: eps_param, iteration: int(i/batch_size)})

    if i % print_info_rate == 0:
        train_error_entropy = error_entropy.eval(feed_dict={
          x: batch_xs, y_: batch_xs})
        train_error_avg_sqr = error_avg_sqr.eval(feed_dict={
          x: batch_xs, y_: batch_xs})
        
        mean_train_error_avg_sqr = mean_train_error_avg_sqr * (1-error_alpha) + train_error_avg_sqr * error_alpha
        iterations.append(i)
        errors.append(train_error_avg_sqr)
        mean_errors.append(mean_train_error_avg_sqr)
        plt_error(fig, ax[:2], iterations, [errors, mean_errors])
        print('%d\t %.4f\t %.4f\t %.4f\t %.4f\t %f' % (i, train_error_entropy, train_error_avg_sqr, mean_train_error_avg_sqr, eps_param, momentum))


In [ ]:
random.seed(1)

test, labels = data_set.get_next_bach(10000)

In [ ]:
result = sess.run(y, feed_dict={x: test, y_: test})

In [ ]:
%matplotlib inline

for i in tnrange(0,100, desc='Pictures'):
    plt.figure()
    compare_results(result[i], test[i], shape, i)

In [ ]:
color_map = list(matplotlib.cm.tab10.colors)
color_map = np.array(color_map)

In [ ]:
numbers = np.argmax(labels, axis=1)
codes = sess.run(code_layer, feed_dict={x: test})

In [ ]:
alpha_scale = 1

In [ ]:
module, theta = cart2pol(codes[:,0], codes[:,1])
module = module**alpha_scale
plot_codes(module, theta, numbers, 'radial2D_'+str(alpha_scale))

In [ ]:
module, theta = cart2pol(codes[:,0], codes[:,1])
module = module**alpha_scale
x_code, y_code = pol2cart(module, theta)
plot_codes(x_code, y_code, numbers, 'normal2D_'+str(alpha_scale))